# Exploration_04 멋진 작사가 만들기


## 목차
  
  - 데이터 준비 및 불러오기
  - 데이터 전처리
  - 평가 데이터셋 분리
  - 인공지능 학습시키기
  - 회고

### step 1. 데이터 준비 및 불러오기
  - 데이터는 노드에 있는 실습(1) 데이터 다듬기에서 제공하는 데이터를 사용한다.
  - glob을 활용해서 모든 txt를 불러오고 raw_corpus 리스트에 문장 단위로 저장한다.

In [1]:
# 데이터 불러오기

import glob
import os, re

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:1])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord"]


### step 2. 데이터 전처리 과정

  - 공백인 문장과 화자가 표기된 문장을 제거해준다.
  - 길이가 0인 문장은 공백인 문장으로 생각할 수 있다.
  - 문장의 끝이 :로 끝나는 경우는 없기 때문에 문장의 끝이 :인 경우 화자가 표기된 문장으로 생각할 수 있다.

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   
    if sentence[-1] == ":": continue  

    if idx > 9: break   
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


  - 텍스트 생성 모델도 단어 사전을 만들어줘야 한다.
  - 문장을 일정한 기준으로 쪼개주는 과정을 토큰화(Tokenize)라고 한다.
  - 가장 심플한 방법은 띄어쓰기를 기준으로 나눠주는 것이다.
  - 토큰화 과정에서 발생하는 몇 가지 문제들이 있다.
      1. Hi, my name is John. *("Hi," "my", ..., "john." 으로 분리됨) - 문장부호  
      2. First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자
      3. He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자
  - 문장부호 문제를 해결하기 위해서 문장부호 양쪽에 공백을 넣어준다.
  - 대소문자 문제를 해결하기 위해서 모든 문자를 소문자로 바꿔준다.
  - 특수문자 문제를 해결하기 위해서 특수문자들을 모두 제거해준다.

In [3]:
# 토큰화를 위한 데이터 전처리 정규 표현식

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


  - 위의 표현식을 통하면 지저분한 문장을 깔끔하게 정리할 수 있다.
  - 위의 표현식을 통해서 <"This @_is ;;;sample        sentence."> 문장이 깔끔하게 정리되서 출력된 것을 확인할 수 있다.
  - 위에서 만든 표현식을 통해서 토큰화를 진행한 후 끝단어 <end>를 제거하면 소스 문장, 첫단어 <start>를 제거하면 타겟문장이 된다.
  - 보통 자연어처리 과정에서 모델의 입력이 되는 문장을 소스문장(source sentence), 정답 역활을 하는 출력문장을 타겟문장(target sentence)이라고 하며 각각 x_train 과 y_train에 해당한다.

In [4]:
# 정제된 문장 모으기

corpus = []

# 공백 혹은 화자가 입력된 문장은 제외하기
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    
    preprocessed_sentence = preprocess_sentence(sentence)
    # 지나치게 긴 문장은 다른 문장들이 과도한 Padding을 가지게 한다.
    # 그렇기 때문에 토큰의 개수가 15개가 넘는 문장은 제외시켜 준다.
    if len(preprocessed_sentence.split(' ')) > 15 : continue 
    corpus.append(preprocessed_sentence)
    
# 정제된 문장 확인하기
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
# tf.keras.preprocessing.text.Tokenizer 패키지를 사용해서 정제된 데이터를 토큰화 해주어야 한다.
# 토큰화 과정을 통해서 단어사전이 만들어지고 데이터가 숫자로 변환된다.
# 이 과정을 벡터화라고 하며 숫자로 변환된 데이터를 텐서라고 한다.

import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 단어장의 크기를 12,000 이상으로 하라고 했기 때문에 12,000으로 설정해준다.
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f60e339c100>


In [6]:
# 토큰화를 통해서 만들어진 사전이 어떻게 구성되어 있는지 확인할 수 있다.

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


  - 만들어진 사전의 인덱스를 통해서 2번이 start라는 것을 알 수 있다.
  - 그렇기 때문에 각 문장의 시작이 start인 2로 시작하고 end인 3으로 끝나는 것을 알 수 있다.

In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


### step 3. 평가 데이터셋 분리

  - Tensor로 변환된 데이터를 사이킷런의 train_test_split( ) 함수를 사용해서 분리해준다.
  - 총 데이터의 20%를 평가 데이터셋으로 사용하라는 조건이 붙었다.

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         random_state=24)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)
Source Val: (31203, 14)
Target Val: (31203, 14)


In [9]:
# 데이터셋 객체 생성
# 텐서플로우는 텐서로 생성된 데이터를 이용해서 tf.data.Dataset 객체를 생성하는 방법을 사용한다.
# tf.data.Dataset.from_tensor_slices() 메서드를 이용해서 tf.data.Dataset 객체를 생성한다.

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1


dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [10]:
valset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
valset = valset.shuffle(BUFFER_SIZE)
valset = valset.batch(BATCH_SIZE, drop_remainder=True)
valset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### step 4. 인공지능 학습시키기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

  - 위 코드에서 embedding_size는 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기로서 값이 커질수록 추상적인 특징을 잡아낼 수 있다.
  - 하지만 충분한 데이터가 주어지지 않으면 옳바른 값을 얻을 수 없다.
  - hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가로 이해할 수 있다.
  - embedding_size와 hidden_size 조절을 통해서 모델을 수정해볼 수 있다.

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 97s 152ms/step - loss: 3.4797
Epoch 2/10
487/487 [==============================] - 82s 168ms/step - loss: 3.0138
Epoch 3/10
487/487 [==============================] - 82s 168ms/step - loss: 2.8530
Epoch 4/10
487/487 [==============================] - 82s 168ms/step - loss: 2.7314
Epoch 5/10
487/487 [==============================] - 82s 168ms/step - loss: 2.6268
Epoch 6/10
487/487 [==============================] - 82s 168ms/step - loss: 2.5323
Epoch 7/10
487/487 [==============================] - 82s 168ms/step - loss: 2.4448
Epoch 8/10
487/487 [==============================] - 82s 168ms/step - loss: 2.3611
Epoch 9/10
487/487 [==============================] - 82s 168ms/step - loss: 2.2814
Epoch 10/10
487/487 [==============================] - 82s 168ms/step - loss: 2.2042


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valset, epochs=10)

Epoch 1/10
487/487 [==============================] - 93s 187ms/step - loss: 2.1821 - val_loss: 2.5213
Epoch 2/10
487/487 [==============================] - 91s 186ms/step - loss: 2.0647 - val_loss: 2.5144
Epoch 3/10
487/487 [==============================] - 90s 185ms/step - loss: 1.9874 - val_loss: 2.5088
Epoch 4/10
487/487 [==============================] - 90s 185ms/step - loss: 1.9160 - val_loss: 2.4962
Epoch 5/10
487/487 [==============================] - 90s 185ms/step - loss: 1.8464 - val_loss: 2.4948
Epoch 6/10
487/487 [==============================] - 90s 184ms/step - loss: 1.7788 - val_loss: 2.4884
Epoch 7/10
487/487 [==============================] - 90s 184ms/step - loss: 1.7130 - val_loss: 2.4834
Epoch 8/10
487/487 [==============================] - 90s 184ms/step - loss: 1.6499 - val_loss: 2.4850
Epoch 9/10
487/487 [==============================] - 90s 184ms/step - loss: 1.5891 - val_loss: 2.4850
Epoch 10/10
466/487 [===========================>..] - ETA: 3s - loss: 1.

In [52]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [41]:
generate_text(model, tokenizer, init_sentence="<start> i m", max_len=20)

'<start> i m a survivor <end> '

In [53]:
generate_text(model, tokenizer, init_sentence="<start> you re", max_len=20)

'<start> you re the only one <end> '

  - embedding_size = 256 / hidden_size = 1024 설정으로 학습데이터의 loss값은 2.2 수준이었지만 val_loss값은 2.4 수준이었다.
  - 현재 모델에서 문장 생성을 했을때는 위의 결과를 얻었지만 다른 여러 문장들을 생성했을때 말이 안되는 문장들이 더 많았다.
  - val_loss값을 좀 더 낮추기 위해서 embedding_size값과 hidden_size값을 수정해서 다시 테스트를 해봐야 할 것 같다.

### Step 4-1. 모델 향상

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512  # 256에서 512로 값을 수정해서 테스트해본다.
hidden_size = 2048    # 1024에서 2048로 값을 수정해서 테스트해본다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 248s 471ms/step - loss: 3.2861
Epoch 2/10
487/487 [==============================] - 230s 473ms/step - loss: 2.7436
Epoch 3/10
487/487 [==============================] - 231s 474ms/step - loss: 2.4347
Epoch 4/10
487/487 [==============================] - 231s 474ms/step - loss: 2.1315
Epoch 5/10
487/487 [==============================] - 231s 474ms/step - loss: 1.8419
Epoch 6/10
487/487 [==============================] - 230s 473ms/step - loss: 1.5825
Epoch 7/10
487/487 [==============================] - 231s 473ms/step - loss: 1.3678
Epoch 8/10
487/487 [==============================] - 231s 474ms/step - loss: 1.2032
Epoch 9/10
487/487 [==============================] - 231s 475ms/step - loss: 1.0917
Epoch 10/10
487/487 [==============================] - 231s 474ms/step - loss: 1.0263


In [18]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valset, epochs=10)

Epoch 1/10
487/487 [==============================] - 254s 518ms/step - loss: 1.0367 - val_loss: 2.1955
Epoch 2/10
487/487 [==============================] - 252s 517ms/step - loss: 0.9801 - val_loss: 2.2399
Epoch 3/10
487/487 [==============================] - 252s 517ms/step - loss: 0.9625 - val_loss: 2.2616
Epoch 4/10
487/487 [==============================] - 252s 516ms/step - loss: 0.9513 - val_loss: 2.2767
Epoch 5/10
487/487 [==============================] - 252s 517ms/step - loss: 0.9432 - val_loss: 2.2932
Epoch 6/10
487/487 [==============================] - 252s 518ms/step - loss: 0.9366 - val_loss: 2.3026
Epoch 7/10
487/487 [==============================] - 252s 517ms/step - loss: 0.9312 - val_loss: 2.3203
Epoch 8/10
487/487 [==============================] - 252s 516ms/step - loss: 0.9269 - val_loss: 2.3256
Epoch 9/10
487/487 [==============================] - 251s 515ms/step - loss: 0.9226 - val_loss: 2.3321
Epoch 10/10
487/487 [==============================] - 252s 516m

  - embedding_size와 hidden_size의 값을 각각 512와 2048로 바꾸어서 테스트를 진행하였다.
  - 하지만 원하는 val_loss 값인 2.2에는 못미치는 2.3의 값을 얻었다.

In [28]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 2048  # 256에서 350으로 값을 수정해서 테스트해본다.
hidden_size = 4096    
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [30]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valset, epochs=10)

Epoch 1/10
487/487 [==============================] - 945s 2s/step - loss: 2.2625 - val_loss: 2.4024
Epoch 2/10
487/487 [==============================] - 944s 2s/step - loss: 1.9316 - val_loss: 2.2877
Epoch 3/10
487/487 [==============================] - 944s 2s/step - loss: 1.6247 - val_loss: 2.2153
Epoch 4/10
487/487 [==============================] - 943s 2s/step - loss: 1.3580 - val_loss: 2.1811
Epoch 5/10
487/487 [==============================] - 943s 2s/step - loss: 1.1589 - val_loss: 2.1909
Epoch 6/10
487/487 [==============================] - 942s 2s/step - loss: 1.0407 - val_loss: 2.2165
Epoch 7/10
487/487 [==============================] - 942s 2s/step - loss: 0.9834 - val_loss: 2.2436
Epoch 8/10
487/487 [==============================] - 942s 2s/step - loss: 0.9560 - val_loss: 2.2520
Epoch 9/10
487/487 [==============================] - 941s 2s/step - loss: 0.9413 - val_loss: 2.2673
Epoch 10/10
487/487 [==============================] - 942s 2s/step - loss: 0.9312 - val_lo

In [35]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [46]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love <end> '

In [47]:
generate_text(model, tokenizer, init_sentence="<start> you love", max_len=20)

'<start> you love <end> '

  - embedding_size와 hidden_size의 값을 각각 2048 / 4096으로 올려서 진행했다.
  - 테스트 시간이 엄청 늘어나서 더 큰 폭으로 올리는 것은 지금 당장 어렵다고 판단했다.
  - 원하는 val_loss값은 2.2 수준의 값을 얻었다.
  - 하지만 문장을 만드는데 있어서 재대로 작동하지 않아서 모델을 다시 학습시켜서 테스트해야했다.

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024  
hidden_size = 2048    
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valset, epochs=10)

Epoch 1/10
487/487 [==============================] - 285s 506ms/step - loss: 3.2096 - val_loss: 2.8442
Epoch 2/10
487/487 [==============================] - 263s 539ms/step - loss: 2.6750 - val_loss: 2.5961
Epoch 3/10
487/487 [==============================] - 265s 544ms/step - loss: 2.3470 - val_loss: 2.4212
Epoch 4/10
487/487 [==============================] - 265s 544ms/step - loss: 2.0271 - val_loss: 2.2874
Epoch 5/10
487/487 [==============================] - 265s 545ms/step - loss: 1.7284 - val_loss: 2.1947
Epoch 6/10
487/487 [==============================] - 266s 545ms/step - loss: 1.4707 - val_loss: 2.1443
Epoch 7/10
487/487 [==============================] - 265s 544ms/step - loss: 1.2663 - val_loss: 2.1241
Epoch 8/10
487/487 [==============================] - 266s 545ms/step - loss: 1.1220 - val_loss: 2.1280
Epoch 9/10
487/487 [==============================] - 266s 545ms/step - loss: 1.0359 - val_loss: 2.1483
Epoch 10/10
487/487 [==============================] - 266s 545m

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20)

'<start> i like the way how you re touchin me <end> '

  - 이번에는 embedding_sze와 hidden_size의 값을 각각 1024와 2048로 낮춰서 진행하였다.
  - 수치가 낮아진 만큼 작업에 걸리는 속도도 많이 단축되었다.
  - val_loss값은 원하는 값보다 더 낮은 2.1의 값을 얻을 수 있었다.
  - 특히 문장을 만드는데 있어서 val_loss 2.2 수준의 모델보다 훨씬 좋은 문장을 만드는 것을 확인할 수 있었다.

### Step 5. 회고

  - 이번 프로젝트는 시간과의 싸움이라고 생각될만큼 모델을 작동시키는데 많은 시간이 소요되었다.
  - 하지만 재미있었던점은 모델의 하이퍼파라미터 변경을 통해서 각각 다른 값을 얻고 또 다른 값에 따라서 모델들의 성능이 차이를 보였다는 것이다.
  - 여기서는 embedding_size와 hidden_size만 변경해주었지만 시간이 좀 더 있었다면 batch_size나 learning_rate, optimizer 변경도 시도해볼만하다고 생각한다.
  - 또 이번 프로젝트를 진행하면서 영어가 아닌 한국어를 통해서도 작사 혹은 작문이 가능한지에 대한 궁금증이 생겼다.
  - 하지만 구글링을 통해서도 파이썬을 통한 한국어 작문에 대한 내용을 발견하기 쉽지 않았다. 추후에 시간이 되면 찾아보면 좋을 거 같다.